In [ ]:
require 'nngraph'
require 'gnuplot'
require 'nn'

checkpoint_1 = torch.load('../../arxiv/all_classes_32_3_MSE.t7')   
a = torch.Tensor(checkpoint_1.train_loss_history) 



In [ ]:
gnuplot.pngfigure('plot_sky_class_temp_1.png')
gnuplot.plot(a)
gnuplot.plotflush()